In [16]:
#!pip install --user opencv-python
import tensorflow as tf

import os
import time
import cv2
import numpy as np
import shutil

from matplotlib import pyplot as plt
from IPython import display

In [17]:
IMAGE_DIR = "cil-cosmology-2020/galaxy-o/galaxy/"
TARGET_DIR = "cil-cosmology-2020/galaxy-o/abstracted/"

In [23]:
image_list = os.listdir(TARGET_DIR)
n_images = 10000

In [20]:
def move_files(filenames, old_directory, new_directory):
    for i in filenames:
        old_file_path = os.path.join(old_directory, str(i))
        new_file_path = os.path.join(new_directory, str(i))

        if not os.path.isdir(new_directory):
            os.mkdir(new_directory)

        # Deletes a file if that file already exists there, you can change this behavior
        if os.path.exists(new_file_path):
            os.remove(new_file_path)
        
        shutil.move(old_file_path, new_file_path)

In [21]:
def abstract_image(img_path, target_dir):
    img_name = os.path.basename(img_path)
    target_path = os.path.join(target_dir, img_name)

    img = cv2.imread(image_path)
    #img = cv2.medianBlur(img,5)
    height, width, _ = img.shape
    
    cimg = np.zeros((height, width, 3), np.uint8)
    #rimg = np.zeros((height, width, 3), np.uint8)
    circles = cv2.HoughCircles(img[:, :, 0],cv2.HOUGH_GRADIENT,1,10,
                            param1=90,param2=5,minRadius=0,maxRadius=0)

    
    if not (circles is None):
        circles = np.uint16(np.around(circles))
    
        for i in circles[0,:]:
            # compute average intensity
            
            x = min(i[1], width-1)
            y = min(i[0], height-1)
            
            color = tuple([int(c) for c in img[x, y]])

            # draw the filled circle
            cv2.circle(cimg,(i[0],i[1]),i[2],color,-1)
            
            # draw the outer circle
            #cv2.circle(rimg,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            #cv2.circle(rimg,(i[0],i[1]),2,(0,0,255),3)
    
    timg = np.ones((height, 2*width, 3), np.uint8)
    timg[:, :width, :] = img[:, :, :]
    timg[:, width:, :] = cimg[:, :, :]
    #timg[:, 2*width:, :] = rimg[:, :, :]
    
    cv2.imwrite(target_path,timg)
    

In [6]:
for i, image_name in enumerate(image_list):
    if i>=n_images:
        break
    print(".", end="")
    if (i+1)%100==0:
                print( f" {i+1}/{n_images}")
    
    image_path = os.path.join(IMAGE_DIR, image_name)
    abstract_image(image_path, TARGET_DIR)

.................................................................................................... 100/10000
.................................................................................................... 200/10000
.................................................................................................... 300/10000
.................................................................................................... 400/10000
.................................................................................................... 500/10000
.................................................................................................... 600/10000
.................................................................................................... 700/10000
.................................................................................................... 800/10000
.................................................................................................... 900/10000
.

In [24]:
# split images into test and training set
test_size = 200
image_array = np.asarray(image_list)
np.random.shuffle(image_array)
move_files(image_array[:test_size], TARGET_DIR, TARGET_DIR+"test")
move_files(image_array[test_size:], TARGET_DIR, TARGET_DIR+"train")